## dotenv and chatmodels setup

In [2]:
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv(find_dotenv(), override=True)

True

In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192", temperature=0.1)

response = llm.invoke("What is the capital of Finland?")
print(response.content)

The capital of Finland is Helsinki.


In [4]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import initialize_agent, Tool, AgentExecutor,create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool

### Template setup

In [5]:
template="""
Answer the question as truthfully as possible.
If you don't know the answer, say "I don't know".
Question: {question}
"""

prompt_template = PromptTemplate.from_template(template)

prompt = hub.pull("hwchase17/react")

#print(type(prompt))

#print(prompt.input_variables)

#print(prompt.template)

c:\Users\Shah\Desktop\langchain-playground\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


### Tools Setup

In [6]:
# Python REPL Tool - for executing Python code
python_repl = PythonREPLTool()

python_repl_tool = Tool(
    name="Python REPL",
    func=python_repl.run,
    description="Useful for when you need to answer questions about Python code. Input should be a valid Python expression."
)

# Wikipedia Tool - for querying Wikipedia
api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)

wikipedia_tool = Tool(
    name="Wikipedia",
    func=wikipedia.run,
    description="Useful for when you need to answer questions about Wikipedia articles. Input should be a valid Wikipedia query."
)

# DuckDuckGo Search Tool - for searching the web
search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name="DuckDuckGo Search",
    func=search.run,
    description="Useful for when you need to answer questions about current events or general knowledge. Input should be a valid search query."
)

tools = [python_repl_tool, wikipedia_tool, duckduckgo_tool]

### Creating Agent

In [7]:
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=3)


### Test out the agent

In [8]:
question = "Generate first 10 numbers of the Fibonacci series"
response = agent_executor.invoke({'input': prompt_template.format(question=question)})
#print(response)



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


Thought: I need to generate the first 10 numbers of the Fibonacci series. This is a mathematical problem, and I can use the Python REPL to solve it.

Action: Python REPL
Action Input: `fibonacci = [0, 1]; [fibonacci.append(fibonacci[-1] + fibonacci[-2]) for _ in range(8)]`Thought: Now that I have generated the first 10 numbers of the Fibonacci series using the Python REPL, I can print the result.

Action: Python REPL
Action Input: `print(fibonacci)`[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]
Thought: I have successfully generated the first 10 numbers of the Fibonacci series using the Python REPL.

Final Answer: The first 10 numbers of the Fibonacci series are [0, 1, 1, 2, 3, 5, 8, 13, 21, 34].

> Finished chain.


In [9]:
question = "What is the capital of Finland?"
response = agent_executor.invoke({'input': prompt_template.format(question=question)})
print(response['output'])



> Entering new AgentExecutor chain...
Thought: I can use Wikipedia to find the answer to this question.

Action: Wikipedia
Action Input: "Finland"

c:\Users\Shah\Desktop\langchain-playground\.venv\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\Shah\Desktop\langchain-playground\.venv\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Page: Finland
Summary: Finland, officially the Republic of Finland, is a Nordic country in Northern Europe. It borders Sweden to the northwest, Norway to the north, and Russia to the east, with the Gulf of Bothnia to the west and the Gulf of Finland to the south, opposite Estonia. Finland has a population of 5.6 million, the majority being ethnic Finns. Its capital and largest city is Helsinki. The official languages are Finnish and Swedish, the mother tongues of 84.1 percent and 5.1 percent of the population, respectively. Finland's climate varies from humid continental in the south to boreal in the north. Its land is predominantly covered by boreal forest, with over 180,000 recorded lakes.
Finland was first settled around 9000 BC after the last Ice Age. During the Stone Age, various cultures emerged, distinguished by different styles of ceramics. The Bronze Age and Iron Ages were marked by contacts with other cultures in Fennoscandia and the Baltic region. From the late 13th century,

In [10]:
question = "Who is current prime minister of UK?"
response = agent_executor.invoke({'input': prompt_template.format(question=question)})
print(response['output'])



> Entering new AgentExecutor chain...
Thought: I need to find the current prime minister of the UK. I can use Wikipedia to find the answer.

Action: Wikipedia
Action Input: "Current Prime Minister of the UK"Page: List of prime ministers of the United Kingdom
Summary: The prime minister of the United Kingdom is the principal minister of the crown of His Majesty's Government, and the head of the British Cabinet. 
There is no specific date for when the office of prime minister first appeared, as the role was not created but rather evolved over time through a merger of duties. The term was regularly, if informally, used by Robert Walpole by the 1730s. It was used in the House of Commons as early as 1805, and it was certainly in parliamentary use by the 1880s, although did not become the official title until 1905, when Arthur Balfour was prime minister.  
Historians generally consider Robert Walpole, who led the government of the Kingdom of Great Britain for over twenty years from 1721, t